In [1]:
#pip3 install google.colab
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

###  Libraries

In [2]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors

import torch

import os
import sys

from MLP import MLP
from Datasets_Functions import *
from Visualization import Visualization
from Evaluations.Evaluation import Evaluator

/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Global Variables

In [3]:
path_to_module = './' # 'C:\\Users\\PAPASOFT INC\\Desktop\\SOFT_SIL'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '1'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### Load datasets

In [4]:
hashMapFilePath = path_to_module 
#updateHashMap("R15",{'batch_size': 50, 'n_clusters': 15, 'module_name': 'datasets.datasets_New', 'function_name': 'load_R15_dataloader', 'option_name': ''},hashMapFilePath)
hashMap = getHashMap(hashMapFilePath)
print(hashMap.keys())
dataset_name = "R3"
datasetProperties = hashMap[dataset_name]
print(datasetProperties)
batch_size = datasetProperties['batch_size'] #= 30
n_clusters = datasetProperties['n_clusters'] #= 3
datasetProperties['module_name'] = 'Datasets.Datasets'
dataloader, input_dim, data_np, labels = functionGetDataset(datasetProperties)
print('Data_Shape is:', input_dim)
print('Batches Number is:',len(dataloader))

dict_keys(['10x73k', 'TCGA', 'coil20', 'eMNIST_balanced_letters', 'eMNIST_mnist', 'eMNIST_balanced_digits', 'Dermatology', 'Ecoil', 'Iris', 'Rings', 'Wine', 'Australian', 'Moons', 'Squeezed_Gauss', 'Gauss_Densities', 'Pendigits', 'fashionMNIST', '3DSpheres', '20_NewsGroups', 'COIL20', 'CIFAR10', 'STL10', 'R15', 'R3'])
{'batch_size': 100, 'n_clusters': 3, 'module_name': 'datasets.datasets_New', 'function_name': 'load_R3_dataloader', 'option_name': ''}
Data_Shape is: 2
Batches Number is: 15


### Parameters

In [5]:
# Batch Size and Number of Clusters
batch_size = datasetProperties['batch_size']
n_clusters = datasetProperties['n_clusters']

# Training Epochs and Learning Rate
lr = 1e-3
n_epochs = 100

kmeans_initialization = True

### Create MLP model

In [6]:
mlp = MLP(device=device, n_clusters=n_clusters, input_dim=input_dim)
mlp.set_training_variables(dataloader=dataloader, batch_size=batch_size, n_epochs=n_epochs, lr=lr)
mlp.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
mlp.set_path()

mlp = mlp.to(device)

### Colors for visualization

In [7]:
color_list = list(mcolors.CSS4_COLORS.keys()) + list(mcolors.XKCD_COLORS.keys())
np.random.shuffle(color_list)
color_list = ['deepskyblue', 'gold', 'hotpink', 'limegreen'] + color_list
visualization = Visualization(color_list,15,mlp)

### Training the MLP

In [8]:
# Initialize the clustering layer using k-means
if(kmeans_initialization): 
    mlp.kmeans_initialization(10)

mlp.train()

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ff04180e200>
Traceback (most recent call last):
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Threadpool

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ff04180fc70>
Traceback (most recent call last):
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Epoch: 0 CL_LOSS: 15.6420 SOFT_SIL: -0.6420 SIL: 0.6577 ACC: 0.88 PUR: 0.88 NMI: 0.72 ARI: 0.71
Epoch: 1 CL_LOSS: 15.6114 SOFT_SIL: -0.6114 SIL: 0.5929 ACC: 0.81 PUR: 0.81 NMI: 0.67 ARI: 0.60
Epoch: 2 CL_LOSS: 15.5937 SOFT_SIL: -0.5937 SIL: 0.5320 ACC: 0.74 PUR: 0.74 NMI: 0.66 ARI: 0.55
Epoch: 3 CL_LOSS: 15.5826 SOFT_SIL: -0.5826 SIL: 0.5153 ACC: 0.72 PUR: 0.72 NMI: 0.66 ARI: 0.55
Epoch: 4 CL_LOSS: 15.5670 SOFT_SIL: -0.5670 SIL: 0.5174 ACC: 0.73 PUR: 0.73 NMI: 0.66 ARI: 0.55
Epoch: 5 CL_LOSS: 15.5548 SOFT_SIL: -0.5548 SIL: 0.5168 ACC: 0.72 PUR: 0.72 NMI: 0.66 ARI: 0.55
Epoch: 6 CL_LOSS: 15.5386 SOFT_SIL: -0.5386 SIL: 0.5124 ACC: 0.72 PUR: 0.72 NMI: 0.66 ARI: 0.55
Epoch: 7 CL_LOSS: 15.5194 SOFT_SIL: -0.5194 SIL: 0.5097 ACC: 0.72 PUR: 0.72 NMI: 0.66 ARI: 0.55
Epoch: 8 CL_LOSS: 15.4997 SOFT_SIL: -0.4997 SIL: 0.4997 ACC: 0.71 PUR: 0.71 NMI: 0.66 ARI: 0.55
Epoch: 9 CL_LOSS: 15.4763 SOFT_SIL: -0.4763 SIL: 0.4914 ACC: 0.70 PUR: 0.70 NMI: 0.66 ARI: 0.55
Epoch: 10 CL_LOSS: 15.4545 SOFT_SIL: -0.

Epoch: 84 CL_LOSS: 15.0016 SOFT_SIL: -0.0016 SIL: -0.0525 ACC: 0.36 PUR: 0.37 NMI: 0.01 ARI: 0.00
Epoch: 85 CL_LOSS: 15.0016 SOFT_SIL: -0.0016 SIL: -0.0753 ACC: 0.37 PUR: 0.38 NMI: 0.01 ARI: 0.01
Epoch: 86 CL_LOSS: 15.0014 SOFT_SIL: -0.0014 SIL: -0.0726 ACC: 0.37 PUR: 0.37 NMI: 0.01 ARI: 0.01
Epoch: 87 CL_LOSS: 15.0020 SOFT_SIL: -0.0020 SIL: -0.0128 ACC: 0.34 PUR: 0.34 NMI: 0.00 ARI: -0.00
Epoch: 88 CL_LOSS: 15.0020 SOFT_SIL: -0.0020 SIL: -0.0398 ACC: 0.40 PUR: 0.40 NMI: 0.02 ARI: 0.02
Epoch: 89 CL_LOSS: 15.0016 SOFT_SIL: -0.0016 SIL: -0.0375 ACC: 0.36 PUR: 0.36 NMI: 0.00 ARI: 0.00
Epoch: 90 CL_LOSS: 15.0017 SOFT_SIL: -0.0017 SIL: -0.0276 ACC: 0.36 PUR: 0.36 NMI: 0.00 ARI: 0.00
Epoch: 91 CL_LOSS: 15.0024 SOFT_SIL: -0.0024 SIL: -0.0106 ACC: 0.34 PUR: 0.34 NMI: 0.00 ARI: -0.00
Epoch: 92 CL_LOSS: 15.0020 SOFT_SIL: -0.0020 SIL: -0.0357 ACC: 0.38 PUR: 0.38 NMI: 0.01 ARI: 0.01
Epoch: 93 CL_LOSS: 15.0017 SOFT_SIL: -0.0017 SIL: -0.0279 ACC: 0.36 PUR: 0.36 NMI: 0.00 ARI: 0.00
Epoch: 94 CL_LOSS:

### $k$-means evaluation

In [9]:
data, labels = mlp.get_data()
kmeans = KMeans(n_clusters=n_clusters, n_init=10).fit(data)
evaluator = Evaluator()
evaluator.evaluate_model(data, labels, kmeans.labels_)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ff0418d8040>
Traceback (most recent call last):
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Threadpool

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ff04180e4d0>
Traceback (most recent call last):
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/geo/anaconda3/envs/gvardakas/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


(0.8953333333333333,
 0.8953333333333333,
 0.7227625615665902,
 0.7310410025034824,
 0.6711373)

In [11]:
visualization.makeExcel()
#visualization.plot_tsne(mlp=True)
visualization.plot(mlp=True)

AttributeError: 'MLP' object has no attribute 'dataDirPath'